In [58]:
import math
import numpy as np
import cv2
from scipy import ndimage
from matplotlib import pyplot as plt

In [ ]:
def detect_points(frame, threshold=0.001, min_distance=5):
    """
    フレームから白い点を検出する。
    """
    # 閾値処理
    _, binary = cv2.threshold(frame, threshold, 1.0, cv2.THRESH_BINARY)
    binary = binary.astype(np.uint8)
    
    # 点のラベリング
    labeled, num_features = ndimage.label(binary)
    
    # 各点の重心を計算
    centers = ndimage.center_of_mass(binary, labeled, range(1, num_features+1))
    centers = np.array(centers)  # (N, 2) の配列 (y, x)
    
    return centers  # 形状: (N, 2)

def compute_motion_vectors(prev_centers, next_centers):
    """
    各点の動きベクトルを計算する。
    最も近い点を対応付けとして動きベクトルを計算する。
    """
    motion_vectors = []
    used_indices = set()
    for p in prev_centers:
        distances = np.linalg.norm(next_centers - p, axis=1)
        min_idx = np.argmin(distances)
        # 一度対応付けられた点は再利用しない
        while min_idx in used_indices and len(used_indices) < len(next_centers):
            distances[min_idx] = np.inf
            min_idx = np.argmin(distances)
        if distances[min_idx] != np.inf:
            used_indices.add(min_idx)
            vector = next_centers[min_idx] - p  # (dy, dx)
            motion_vectors.append(vector)
        else:
            # 対応する点が見つからない場合
            motion_vectors.append(np.array([0.0, 0.0]))
    return np.array(motion_vectors)  # 形状: (N, 2)

def warp_frame(frame, centers, motion_vectors, alpha):
    """
    フレームをワープ（変形）させる。
    各点の位置を動きベクトルに基づいて補間位置に移動させる。
    """
    warped_frame = np.copy(frame)
    height, width = frame.shape
    
    for (center, vector) in zip(centers, motion_vectors):
        # 補間位置を計算
        interp_pos = center + alpha * vector  # (y, x)
        y, x = interp_pos
        y = int(round(y))
        x = int(round(x))
        if 0 <= y < height and 0 <= x < width:
            # 元の点の位置
            orig_y, orig_x = center
            orig_y = int(round(orig_y))
            orig_x = int(round(orig_x))
            if 0 <= orig_y < height and 0 <= orig_x < width:
                # ピクセル値を移動
                warped_frame[y, x] = frame[orig_y, orig_x]
    return warped_frame

def interpolate_frames(frames, num_interpolations=1):
    """
    フレーム補間を行う。
    frames: (num_frames, height, width) の配列
    num_interpolations: 各フレーム間に挿入するフレーム数
    """
    interpolated = []
    num_frames, height, width = frames.shape
    
    for t in range(num_frames - 1):
        current_frame = frames[t]
        next_frame = frames[t+1]
        
        # 点の検出
        current_centers = detect_points(current_frame)
        next_centers = detect_points(next_frame)
        
        if len(current_centers) == 0 or len(next_centers) == 0:
            # 点が検出されない場合は単純にフレームをコピー
            interpolated.append(current_frame)
            continue
        
        # 動きベクトルの計算
        motion_vectors = compute_motion_vectors(current_centers, next_centers)
        if t == 32:
            print(motion_vectors)
        
        interpolated.append(current_frame)
        
        for k in range(1, num_interpolations + 1):
            alpha = k / (num_interpolations + 1)
            
            # 現在のフレームをワープ
            warped_current = warp_frame(current_frame, current_centers, motion_vectors, alpha)
            # 次のフレームをワープ
            warped_next = warp_frame(next_frame, next_centers, -motion_vectors, 1 - alpha)
            
            # ブレンディング
            interp_frame = (1 - alpha) * warped_current + alpha * warped_next
            # 正規化
            # interp_frame = np.clip(interp_frame, 0.0, 1.0)
            interpolated.append(interp_frame)
    
    interpolated.append(frames[-1])
    return np.array(interpolated)

In [59]:
# DATA_PATH = "../data"
DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
H_SETTING = "hadamard_FISTA_p-5_lmd-1_m-128"
frames = np.load(f"{DATA_PATH}/241022/systemMatrix/H_tensor_{H_SETTING}.npy")

In [ ]:
# フレーム補間
desired_total_frames = 255
num_original_frames = 128
interpolated_frames = interpolate_frames(frames, num_interpolations=1)
print(f"Original frames: {frames.shape}")
print(f"Interpolated frames: {interpolated_frames.shape}")


In [ ]:
# 表示したいフレームの範囲を設定
start_index = 32
end_index = 47
frames_to_display = interpolated_frames[start_index:end_index + 1]

num_frames = len(frames_to_display)
cols = 4  # 列数を設定（必要に応じて変更可能）
rows = math.ceil(num_frames / cols)  # 行数を自動計算

# サブプロットを作成
fig, axs = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))

# サブプロットの軸を1次元にフラット化してループ処理
for idx, ax in enumerate(axs.flat):
    if idx < num_frames:
        frame = frames_to_display[idx]
        img = ax.imshow(frame, cmap='gray')
        plt.colorbar(img, ax=ax)
        ax.set_title(f'Frame {start_index + idx}')
    else:
        ax.axis('off')  # 使用しないサブプロットを非表示に

# import imageio
# frames_to_save = ((interpolated_frames / 0.02) * 255).astype(np.uint8)
# imageio.mimsave('interpolated_video.gif', frames_to_save, fps=30)
# print("Interpolated video saved as 'interpolated_video.gif'.")